In [2]:
!pip install -qq transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [3]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
from datasets import load_dataset
import torch

In [4]:
urbansound_dataset = load_dataset("danavery/urbansound8K")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8732 [00:00<?, ? examples/s]

In [5]:
feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [6]:
from tqdm.notebook import tqdm

samples_16k = []
for i in tqdm(range(len(urbansound_dataset['train']))):
    if urbansound_dataset['train'][i]['audio']['sampling_rate'] == 16000:
        samples_16k.append(urbansound_dataset['train'][i])

  0%|          | 0/8732 [00:00<?, ?it/s]

In [7]:
len(samples_16k)

45

In [59]:
def get_sample(i):
    return feature_extractor(samples_16k[i]["audio"]["array"][:500],
                             sampling_rate=samples_16k[i]["audio"]["sampling_rate"],
                             return_tensors="pt")

In [60]:
samples_16k[20]

{'audio': {'path': '187863-4-0-0.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.14251709,
         -0.10592651, -0.06546021]),
  'sampling_rate': 16000},
 'slice_file_name': '187863-4-0-0.wav',
 'fsID': 187863,
 'start': 0.0,
 'end': 4.0,
 'salience': 1,
 'fold': 10,
 'classID': 4,
 'class': 'drilling'}

In [61]:
sample = get_sample(20)

In [62]:
sample['input_values'].shape

torch.Size([1, 1024, 128])

In [47]:
with torch.no_grad():
    logits = model(**sample).logits

In [48]:
top_classes = torch.argsort(logits, dim=-1, descending=True).flatten()[:10]
top_labels = [model.config.id2label[id.item()] for id in top_classes]
top_labels

['Steam',
 'Hiss',
 'Vehicle',
 'Jet engine',
 'Train',
 'Steam whistle',
 'Sound effect',
 'Speech',
 'Music',
 'Rail transport']

Listen to samples: https://www.kaggle.com/datasets/chrisfilo/urbansound8k

Model: https://huggingface.co/MIT/ast-finetuned-audioset-10-10-0.4593

Other possible models:

- https://huggingface.co/thelou1s/panns-inference

- https://huggingface.co/ALM/wav2vec2-base-audioset